## Inspect Trained Model


In [72]:
import math
import os

import matplotlib.pyplot as plt
import torch
import yaml
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
from torch.utils.data import Dataset

import data
from networks import *
from utils import mse2psnr

### Select model to inspect


In [73]:
# exp_name = "non_continual"
# exp_name = "continual_base"
# exp_name = "l1/l1_1e-4"
# exp_name = "sharpen/sharpen_first_log"
exp_name = "ewc/ewc_1000"

date = "2023.01.06"
ids = [dir for dir in os.listdir(f"exp/{exp_name}/{date}") if not dir.startswith(".")]

# Take the first ID under this date
id = ids[0]
# Or specify ID
# id = "234224"

path = f"exp/{exp_name}/{date}/{id}"

config_path = f"{path}/.hydra/config.yaml"
ckpt_path = f"{path}/ckpt/final.pt"

In [74]:
# Load config
with open(config_path, "r") as stream:
    config = yaml.safe_load(stream)
    cfg = DictConfig(config)

# Load model
model = instantiate(cfg.network)
ckpt = torch.load(ckpt_path)
model_state_dict = ckpt["model_state_dict"]
model.load_state_dict(ckpt["model_state_dict"])
model.eval()

# Load source image
dataset = instantiate(cfg.data)

### Sanity checks


In [75]:
def eval(model_to_eval, eval_coords, show_img_out):
    if eval_coords == "full":
        model_input = dataset.full_coords
        ground_truth = dataset.full_pixels
    else:
        model_input = dataset.coords_regions[eval_coords]
        ground_truth = dataset.pixels_regions[eval_coords]

    model_output = model_to_eval(model_input)[0]
    side_length = int(math.sqrt(model_output.shape[0]))

    mse = ((model_output - ground_truth) ** 2).mean().item()
    psnr = mse2psnr(mse)

    # Recover spatial dimension for visualization
    img_out = (
        model_output.cpu().view(side_length, side_length, -1).detach()
    )

    # Clamp image in [0, 1] for visualization
    img_out = torch.clip(img_out, 0, 1)
    if show_img_out:
        fig = plt.figure()
        plt.imshow(img_out)

    return mse, psnr, img_out

mse, psnr, img_out = eval(model, eval_coords="full", show_img_out=False)
print(f"full_mse={mse}")
print(f"full_psnr={psnr}")

full_mse=0.021260440349578857
full_psnr=16.724277445440634


In [76]:
for region in range(dataset.num_regions):
    _, psnr, _ = eval(model, eval_coords=region, show_img_out=False)
    print(f"psnr_region{region}={psnr}")

psnr_region0=14.302002629953305
psnr_region1=15.697775143835589
psnr_region2=16.866288225801995
psnr_region3=33.98224504547916


### Weight Sparsity

In [77]:
def get_model_weight_sparsity(model):
    total_params = 0
    sparse_params = 0
    for param in model.parameters():
        total_params += torch.numel(param)
        sparse_params += torch.isclose(param, torch.zeros_like(param)).sum()
    return sparse_params / total_params

print(f"sparsity={get_model_weight_sparsity(model)}")

sparsity=0.0


### Prune Weights


In [78]:
from torch.nn.utils import prune

In [79]:
def get_module_names(model):
    module_names = []
    for param_name in dict(model.named_parameters()).keys():
        # Module name is everything in param name except for the last word
        # e.g. If param name is 'net.0.linear.weight', then module name is 'net.0.linear'
        name = '.'.join(param_name.split('.')[:-1])
        module_names.append(name)
    return module_names

def get_prunable_params(model):
    module_param_pairs = []
    module_names = get_module_names(model)
    for module_name in module_names:
        module = dict(model.named_modules())[module_name]
        module_param_pairs.append((module, "weight"))
        module_param_pairs.append((module, "bias"))
    
    return module_param_pairs

prunable_params = get_prunable_params(model)

In [80]:
def apply_pruning(prunable_params, finalize_pruning):
    for module, prunable_param_name in prunable_params:
        prune.l1_unstructured(module, prunable_param_name, amount=0.004)
        if finalize_pruning:
            prune.remove(module, prunable_param_name)

apply_pruning(prunable_params, True)
print(f"sparsity (after pruning)={get_model_weight_sparsity(model)}")

sparsity (after pruning)=0.00399668188765645


In [81]:
mse, psnr, img_out = eval(model, True)
print(f"mse={mse}")
print(f"psnr={psnr}")

TypeError: eval() missing 1 required positional argument: 'show_img_out'

Conclusion: A sparse SIREN can still reconstruct original image very well, but we cannot get such model by training with L1 penalty!